In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms, utils
from torchvision.transforms import Resize, ToTensor, Normalize
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler

from PIL import Image
from tqdm.auto import tqdm
from facenet_pytorch import MTCNN, InceptionResnetV1

from sklearn.metrics import f1_score

In [2]:
TRAIN_PATH = '/opt/ml/input/data/train/'
TRAIN_IMAGE_PATH = '/opt/ml/input/data/train/processed_train_images/'

In [3]:
train_df = pd.read_csv(os.path.join(TRAIN_PATH, 'preprocessed_train.csv'))
train_df

,Unnamed: 0,id,gender,mask,age,age_3,age_11,label,path
0,0,004394,0,1,53,1,5,7,/opt/ml/input/data/train/processed_train_image...
1,1,000545,1,2,58,1,5,16,/opt/ml/input/data/train/processed_train_image...
2,2,006155,0,0,20,0,2,0,/opt/ml/input/data/train/processed_train_image...
3,3,006236,0,1,20,0,2,6,/opt/ml/input/data/train/processed_train_image...
4,4,003713,1,0,55,1,5,4,/opt/ml/input/data/train/processed_train_image...
...,...,...,...,...,...,...,...,...,...
18888,18888,003314,1,0,20,0,2,3,/opt/ml/input/data/train/processed_train_image...
18889,18889,006955,0,0,19,0,1,0,/opt/ml/input/data/train/processed_train_image...
18890,18890,003555,1,0,48,1,4,4,/opt/ml/input/data/train/processed_train_image...
18891,18891,003469,1,2,55,1,5,16,/opt/ml/input/data/train/processed_train_image...


In [4]:
class maskDataset(Dataset):
    def __init__(self, train_df, transform, train=True):
        self.train = train
        self.transform = transform

        if self.train == True:
            self.paths = train_df['path']
            self.labels = train_df['label']
        else:
            self.paths = train_df
        

    def __getitem__(self, index):
        if self.train:
            image = Image.open(self.paths.iloc[index])
            label = self.labels.iloc[index]
        else:
            image = Image.open(self.paths[index])

        if self.transform:
            image = self.transform(image)

        if self.train:
            return image, torch.tensor(label)
        else:
            return image

    def __len__(self):
        return len(self.paths)

In [5]:
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    #Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2))
])

In [6]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train_df, test_size=0.2, shuffle=True, stratify=train_df['label'], random_state=34)

In [7]:
BATCH_SIZE = 16

### - 한번만 설치

pip install ipywidgets                 
jupyter nbextension enable --py widgetsnbextension

In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ('My Device :', device)

detector = MTCNN(device=device, post_process=False)
tf = transforms.ToPILImage()

train_dataset = maskDataset(train, transform)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=1)

valid_dataset = maskDataset(valid, transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=1)

resnet = InceptionResnetV1(pretrained='vggface2', num_classes=18, classify=True)
resnet.to(device)

LEARNING_RATE = 1e-4
NUM_EPOCH = 1

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=LEARNING_RATE)

dataloaders = {
    "train" : train_dataloader,
    "valid" : valid_dataloader
}

My Device : cuda:0


In [9]:
best_acc = 0
best_loss = 0

transforom_one = transforms.CenterCrop((160, 160))

for epoch in tqdm(range(NUM_EPOCH)):
    for phase in ['train', 'valid']:
        running_loss = 0
        running_acc = 0

        if phase == 'train':
            resnet.train()
        elif phase == 'valid':
            resnet.eval()

        for idx, (images, labels) in enumerate(dataloaders[phase]):
            images = images.to(device)
            labels = labels.to(device)

            faces = []
            for img in images:
                face = detector(tf(img))
                if face == None:
                    face = transforom_one(img)
                face = face.to(device)
                faces.append(face)
            

            faces = torch.stack(faces, dim=0)
            faces = faces.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                logits = resnet(faces)
                _, pred = torch.max(logits, 1)
                print('pred:',pred)
                loss = loss_fn(logits, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * images.size(0)
            running_acc += torch.sum(pred == labels.data)
            

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)


        print (f'[{phase}] 현재 EPOCH : {epoch}, loss : {epoch_loss}, acc : {epoch_acc}')

pred: tensor([ 5,  9, 17,  6,  0, 11, 14,  9,  5,  2,  7,  5,  6,  1,  0,  0],
       device='cuda:0')
pred: tensor([ 7, 17, 16, 10, 17, 14,  7,  0,  4,  4, 10, 13,  0,  7,  7, 10],
       device='cuda:0')
pred: tensor([10,  0,  0,  4, 12,  4,  4, 12, 16,  4,  5, 12,  4,  4,  6, 14],
       device='cuda:0')
pred: tensor([17,  7,  4,  4, 16, 14, 10,  0,  9,  3,  1,  4,  5,  1,  4,  8],
       device='cuda:0')
pred: tensor([ 4, 14,  0,  7,  4,  2,  0,  6,  1, 16,  2, 16,  3,  4,  5,  4],
       device='cuda:0')
pred: tensor([10,  4,  7,  4, 17,  0, 11,  4,  3, 10,  7,  3,  7,  4,  9, 16],
       device='cuda:0')
pred: tensor([16,  4,  3,  0,  4,  6,  0,  0,  4, 16,  3,  0,  4,  0, 11,  1],
       device='cuda:0')



KeyboardInterrupt: 

In [ ]:
TEST_DIR = '/opt/ml/input/data/eval/'

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다
submission = pd.read_csv(os.path.join(TEST_DIR, 'info.csv'))
image_dir = os.path.join(TEST_DIR, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
dataset = maskDataset(image_paths, transform, train=False)
loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
resnet.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in tqdm(loader):
    images = images.to(device)
    faces = []
    for img in images:
        temp = tf(img)
        face = detector(tf(img))
        if face == None:
            face = transforom_one(img)
        face = face.to(device)
        faces.append(face)

    faces = torch.stack(faces, dim=0)
    faces = faces.to(device)
    with torch.no_grad():
        logits = resnet(images)
        _, pred = torch.max(logits, 1)
        # pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(TEST_DIR, 'submission.csv'), index=False)
print('test inference is done!')

pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], device='cuda:0')
pred = tensor([1168], dev

KeyboardInterrupt: 

In [ ]:
sorted(submission['ans'].unique())

[3]